In [1]:
import spacy
import re
import string
import random
from spacy import util
from spacy.tokens import Doc
from spacy.training import Example
from spacy.language import Language
import json

In [2]:
!python3 -m spacy download en_core_web_lg

2022-12-13 03:10:41.055035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.7/lib64:/shared/centos7/anaconda3/2021.07-TF/lib:/shared/centos7/nodejs/14.15.4/lib
2022-12-13 03:10:41.055125: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 587.7 MB 4.1 kB/s  eta 0:00:01   |▊                               | 13.5 MB 2.9 MB/s eta 0:03:21     |████                            | 71.5

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
f = open('annotations.json')
data = json.load(f)
train_data = data['annotations']


# Disable all pipe components except 'ner'
disabled_pipes = []
for pipe_name in nlp.pipe_names:
    if pipe_name != 'ner':
        nlp.disable_pipes(pipe_name)
        disabled_pipes.append(pipe_name)

optimizer = nlp.create_optimizer()
for _ in range(25):
    random.shuffle(train_data)
    for raw_text, entity_offsets in train_data:
        doc = nlp.make_doc(raw_text)
        example = Example.from_dict(doc, {"entities": entity_offsets['entities']})
        nlp.update([example], sgd=optimizer)

# Enable all previously disabled pipe components
for pipe_name in disabled_pipes:
    nlp.enable_pipe(pipe_name)

# Result after training


print(nlp.get_pipe("ner").labels)


('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GENRE', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'TITLE', 'WORK', 'WORK_OF_ART')


In [6]:
import pandas as pd
import csv
df = pd.read_csv('imdb_data.csv')
df

,docid,title,year,type,colors,genres,languages,countries,releasedates,directors,producers,composers,cast
0,540030,The American Film Institute Salute to Clint Ea...,1996.0,TelevisionMovie,['Color'],['Documentary'],['English'],['USA'],{'USA': '1 May 1996'},"['Horvitz, Louis J.']","['Seligman, Michael B.', 'Herman, Danette', 'S...",[],"{'Beatty, Warren': 'Himself', 'Brosnan, Pierce..."
1,26630,Ai qing amoeba,1997.0,movie,['Color'],['Comedy'],['Cantonese'],['Hong Kong'],{'Hong Kong': '27 February 1997'},"['Shu, Kei']",[],[],"{'Cheung, Julian': 'Mo Yan Dung', 'Cheung, Tat..."
2,164371,Earth to Kids: A Guide to Products for a Healt...,1990.0,TelevisionMovie,['Color'],['Family'],['English'],['USA'],{},[],"['Markowitz, Susan', 'Howie, Betsy', 'Newman, ...",[],"{'Fyfe, Jim': 'Host'}"
3,594507,The Sweater,1980.0,movie,['Color'],"['Family', 'Short', 'Sport', 'Animation', 'Bio...","['French', 'English']",['Canada'],{},"['Cohen, Sheldon (II)']","['Lamb, Derek (I)', 'Canell, Marrin', 'Verrall...","['Roger, Normand']","{'Carrier, Roch': 'Narrator', 'Moreau, Jean-Gu..."
4,227582,Half Nelson,1985.0,TelevisionMovie,['Color'],"['Drama', 'Comedy']",['English'],['USA'],{'USA': '24 March 1985'},"['Seidelman, Arthur Allan']",[],[],"{'Brittany, Morgan': 'Herself', 'Calhoun, Rory..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114374,82821,Bridal Bail,1935.0,movie,['Black and White'],"['Short', 'Comedy']",['English'],['USA'],{'USA': '1935'},"['Stevens, George (I)']","['Brock, Lou (I)']",[],"{'Appel, Sam': 'Himself', 'Brewster, June': 'H..."
114375,497765,Simply Sophia,2008.0,DirectToVideo,['Color'],"['Drama', 'Short']",['English'],['USA'],{'USA': '11 August 2008'},"['McMahand, Jon']","['Bledsoe, Debbie', 'McMahand, Jon']",[],"{'Anderson, Quinne': 'Waitress', 'Bledsoe, Deb..."
114376,556893,The Farewell Gift,2008.0,movie,['Color'],"['Drama', 'Short']",['English'],['USA'],{'USA': '17 December 2008'},"['Sen, Arani']","['Sen, Arani']",[],"{'Edwards, Jay (IV)': 'Tony Gemini', 'Gervason..."
114377,292446,King Lines,2007.0,movie,['Color'],"['Sport', 'Adventure']",['English'],"['Canada', 'USA']","{'Canada': '6 February 2008', 'USA': '19 Octob...","['Lowell, Josh', 'Mortimer, Peter']","['Lowell, Josh', 'Mortimer, Peter']",[],"{'LaCasse, Melissa': 'Herself - Climber', 'Lam..."


In [7]:
doc = []
doc_noun = []

def get_column_tokenized(column_name):
    for i in range(len(df[column_name])):
        doc.append(list(nlp(df[column_name][i])))
        temp = nlp(df[column_name][i])
        spacy.displacy.render(temp, style = "ent")
        doc_noun.append(list(temp.noun_chunks))
    #print(doc)
    print(doc_noun)

In [8]:
nlp.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GENRE',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'TITLE',
 'WORK',
 'WORK_OF_ART')

In [9]:
Jaccardy_indexes_for_all_queries={}

In [10]:
def get_deterministic_results():
    ans=df
    while(True):
        print(f"1. Title \n2. Release Year \n3. Genre \n4. Languages \n5. Shot in Countries \n6. Directors \n7. Producers \n8. Composers \n9. Cast \n0. Show Results \nChoose an option \n")
        choice=int(input())
        if choice == 1:
            text=input("Enter the title: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.title.apply(lambda x: j in x.lower())]
        elif choice == 2:
            text=input("Enter the Release year: ")
            ans = ans[ans['year'] == float(text)]
        elif choice == 3:
            text=input("Enter the Genre: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.genres.apply(lambda x: j in x.lower())]
        elif choice == 4:
            text=input("Enter the Languages: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.languages.apply(lambda x: j in x.lower())]
        elif choice == 5:
            text=input("Enter the Countries: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.countries.apply(lambda x: j in x.lower())]
        elif choice == 6:
            text=input("Enter the Directors: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.directors.apply(lambda x: j in x.lower())]
        elif choice == 7:
            text=input("Enter the Producers: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.producers.apply(lambda x: j in x.lower())]
        elif choice == 8:
            text=input("Enter the Composers: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.composers.apply(lambda x: j in x.lower())]
        elif choice == 9:
            text=input("Enter the Cast: ")
            temp=text.lower().split(' ')
            for j in temp:
                ans = ans[ans.cast.apply(lambda x: j in x.lower())]
        elif choice == 0:
            break
    return ans
    


In [142]:
def jaccard_index(df1,df2):
    if df1.empty or df2.empty:
        return 0.0
    df_int=pd.merge(df1,df2)
    df_union=pd.concat([df1, df2], axis=1)
    jac_index=len(df_int)/(len(df_union))
    return jac_index

In [12]:
def accuracy(index):
    val=sum(index.values())
    acc=val/len(index)
    return acc

In [205]:

def filter_result(li, result=df):
    work = ''
    for i in li:
        #print(work)
        if i.ent_type_.lower() == 'work':
            work = i.text.lower()
        if not result.empty:
            if i.ent_type_.lower() == 'language':
                result = result[result.languages.apply(lambda x: i.text.lower() in x.lower())]
            elif i.ent_type_.lower() == 'gpe':
                result = result[result.countries.apply(lambda x: i.text.lower() in x.lower())]
            elif i.ent_type_.lower() == 'title' and i.text!="'":
                result = result[result.title.str.contains(i.text, na=False, case=False)]
            elif i.ent_type_.lower() == 'genre':
                result = result[result.genres.apply(lambda x: i.text.lower() in x.lower())]
            elif i.ent_type_.lower() == 'date':
                result = result.loc[df['year'] == float(i.text)]
            elif i.ent_type_.lower() == 'person':
                word=str(li)[:str(li).index(i.text)]
                if 'direct' in word.lower() or 'produc' in word.lower() or 'compos' in word.lower():
                    if 'direct' in work:
                        result = result[result.directors.apply(lambda x: i.text.lower() in x.lower())]
                    if 'produc' in work:
                        result = result[result.producers.apply(lambda x: i.text.lower() in x.lower())]
                    if 'compos' in work:
                        result = result[result.composers.apply(lambda x: i.text.lower() in x.lower())]
                else:
                    result = result[result.cast.apply(lambda x: i.text.lower() in x.lower())]
        else:
            print("No results were found for the given query. Please try again! :)")
            break
    return result


In [213]:
query = "Give me a movie in English directed by James Cameron"
li = nlp(query)
spacy.displacy.render(li, style = "ent")
# print(li)

In [214]:
result = filter_result(li)
result

,docid,title,year,type,colors,genres,languages,countries,releasedates,directors,producers,composers,cast
8382,214413,Ghosts of the Abyss,2003.0,movie,['Color'],"['Documentary', 'Family', 'Short']",['English'],['USA'],"{'UK': '18 April 2003', 'Thailand': '5 March 2...","['Cameron, James (I)']","['Steenolsen, Chris', 'Marsh, Ed W.', 'Rackaus...","['McNeely, Joel', 'Torban, Lisa']","{'Abernathy, Lewis': 'Himself', 'Arneson, Char..."
12365,535888,Terminator 2: Judgment Day,1991.0,movie,['Color'],"['Thriller', 'Action', 'Sci-Fi']","['Spanish', 'English']","['France', 'USA']","{'Sweden': '13 September 1991', 'Czechoslovaki...","['Cameron, James (I)']","['Austin, Stephanie', 'Rack, B.J.', 'Hurd, Gal...","['Fiedel, Brad']","{'Abbott, Dalton': 'Infant John Connor', 'Alba..."
20189,52388,Avatar,2009.0,movie,['Color'],"['Thriller', 'Adventure', 'Action', 'Sci-Fi']",['English'],['USA'],"{'Sweden': '18 December 2009', 'Hong Kong': '1...","['Cameron, James (I)']","['Tashjian, Janace', 'Breton, Brooke', 'McLagl...","['Horner, James']","{'Alonso, Laz': ""Tsu'Tey"", 'Clover, Amy': 'Nav..."
22227,595350,The Terminator,1984.0,movie,['Color'],"['Thriller', 'Action', 'Sci-Fi']","['Spanish', 'English']","['UK', 'USA']","{'Sweden': '8 February 1985', 'Czechoslovakia'...","['Cameron, James (I)']","['Daly, John (III)', 'Gibson, Derek (II)']","['Fiedel, Brad']","{'Biehn, Michael': 'Kyle Reese', 'Boen, Earl':..."
27338,127072,Dark Angel,2000.0,TelevisionSeries,['Color'],"['Adventure', 'Action', 'Sci-Fi']",['English'],['USA'],"{'Sweden': '7 January 2002', 'Argentina': '11 ...","['Anderson, Sarah Pia', 'Beggs, Morgan', 'Bill...","['Lamar, Gina', 'Biller, Kenneth', 'David, Mar...","['Smith, Amani K.']","{'Abbott, David (I)': 'Minister', 'Aberle, Ste..."
43028,433722,Piranha Part Two: The Spawning,1981.0,movie,['Color'],"['Drama', 'Romance', 'Horror', 'Sci-Fi']",['English'],"['Italy', 'USA']","{'Sweden': '25 December 1982', 'Portugal': 'Fe...","['Assonitis, Ovidio G.', 'Cameron, James (I)']","['Assonitis, Ovidio G.', 'Leeuwen, Chako van',...","['Cipriani, Stelvio']","{'Berg, Tracey (I)': 'Beverly', 'Brewer Jr., C..."
43666,61046,Battle Angel,2011.0,movie,['Color'],"['Thriller', 'Romance', 'Action', 'Sci-Fi']",['English'],['USA'],{'USA': '2011'},"['Cameron, James (I)']","['Cameron, James (I)', 'Landau, Jon (I)']",[],{}
61337,30165,Aliens,1986.0,movie,['Color'],"['Thriller', 'Adventure', 'Horror', 'Action', ...",['English'],"['UK', 'USA']","{'Uruguay': '2 January 1987', 'Sweden': '3 Oct...","['Cameron, James (I)']","['Carroll, Gordon', 'Giler, David', 'Hill, Wal...","['Horner, James']","{'Armstrong, William (I)': 'Lydecker', 'Biehn,..."
74468,30189,Aliens of the Deep,2005.0,movie,['Color'],['Documentary'],['English'],['USA'],"{'Hungary': '20 September 2005', 'Argentina': ...","['Cameron, James (I)', 'Quale, Steven']","['Debiec, Christopher A.', 'Marsh, Ed W.', 'Ca...","['Hwang, Jeehun']","{'Atkins, Dr. Michael': 'Himself, Science obse..."
79965,236635,Hercules: The Legendary Journeys,1995.0,TelevisionSeries,['Color'],"['Drama', 'Adventure', 'Action', 'Fantasy']",['English'],"['New Zealand', 'USA']","{'Portugal': '8 February 1997', 'Netherlands':...","['Beesley, Mark (I)', 'Bond, Timothy', 'Braver...","['Joyce, Bernadette', 'Bielak, Robert', 'Gruen...","['LoDuca, Joseph']","{'Ackerley, Rachel': 'Evander', 'Acorn, Jonath..."


In [201]:
answer=get_deterministic_results()
answer 

1. Title 
2. Release Year 
3. Genre 
4. Languages 
5. Shot in Countries 
6. Directors 
7. Producers 
8. Composers 
9. Cast 
0. Show Results 
Choose an option 



 4
Enter the Languages:  english


1. Title 
2. Release Year 
3. Genre 
4. Languages 
5. Shot in Countries 
6. Directors 
7. Producers 
8. Composers 
9. Cast 
0. Show Results 
Choose an option 



 6
Enter the Directors:  james cameron


1. Title 
2. Release Year 
3. Genre 
4. Languages 
5. Shot in Countries 
6. Directors 
7. Producers 
8. Composers 
9. Cast 
0. Show Results 
Choose an option 



 0


,docid,title,year,type,colors,genres,languages,countries,releasedates,directors,producers,composers,cast
8382,214413,Ghosts of the Abyss,2003.0,movie,['Color'],"['Documentary', 'Family', 'Short']",['English'],['USA'],"{'UK': '18 April 2003', 'Thailand': '5 March 2...","['Cameron, James (I)']","['Steenolsen, Chris', 'Marsh, Ed W.', 'Rackaus...","['McNeely, Joel', 'Torban, Lisa']","{'Abernathy, Lewis': 'Himself', 'Arneson, Char..."
12365,535888,Terminator 2: Judgment Day,1991.0,movie,['Color'],"['Thriller', 'Action', 'Sci-Fi']","['Spanish', 'English']","['France', 'USA']","{'Sweden': '13 September 1991', 'Czechoslovaki...","['Cameron, James (I)']","['Austin, Stephanie', 'Rack, B.J.', 'Hurd, Gal...","['Fiedel, Brad']","{'Abbott, Dalton': 'Infant John Connor', 'Alba..."
20189,52388,Avatar,2009.0,movie,['Color'],"['Thriller', 'Adventure', 'Action', 'Sci-Fi']",['English'],['USA'],"{'Sweden': '18 December 2009', 'Hong Kong': '1...","['Cameron, James (I)']","['Tashjian, Janace', 'Breton, Brooke', 'McLagl...","['Horner, James']","{'Alonso, Laz': ""Tsu'Tey"", 'Clover, Amy': 'Nav..."
22227,595350,The Terminator,1984.0,movie,['Color'],"['Thriller', 'Action', 'Sci-Fi']","['Spanish', 'English']","['UK', 'USA']","{'Sweden': '8 February 1985', 'Czechoslovakia'...","['Cameron, James (I)']","['Daly, John (III)', 'Gibson, Derek (II)']","['Fiedel, Brad']","{'Biehn, Michael': 'Kyle Reese', 'Boen, Earl':..."
27338,127072,Dark Angel,2000.0,TelevisionSeries,['Color'],"['Adventure', 'Action', 'Sci-Fi']",['English'],['USA'],"{'Sweden': '7 January 2002', 'Argentina': '11 ...","['Anderson, Sarah Pia', 'Beggs, Morgan', 'Bill...","['Lamar, Gina', 'Biller, Kenneth', 'David, Mar...","['Smith, Amani K.']","{'Abbott, David (I)': 'Minister', 'Aberle, Ste..."
43028,433722,Piranha Part Two: The Spawning,1981.0,movie,['Color'],"['Drama', 'Romance', 'Horror', 'Sci-Fi']",['English'],"['Italy', 'USA']","{'Sweden': '25 December 1982', 'Portugal': 'Fe...","['Assonitis, Ovidio G.', 'Cameron, James (I)']","['Assonitis, Ovidio G.', 'Leeuwen, Chako van',...","['Cipriani, Stelvio']","{'Berg, Tracey (I)': 'Beverly', 'Brewer Jr., C..."
43666,61046,Battle Angel,2011.0,movie,['Color'],"['Thriller', 'Romance', 'Action', 'Sci-Fi']",['English'],['USA'],{'USA': '2011'},"['Cameron, James (I)']","['Cameron, James (I)', 'Landau, Jon (I)']",[],{}
61337,30165,Aliens,1986.0,movie,['Color'],"['Thriller', 'Adventure', 'Horror', 'Action', ...",['English'],"['UK', 'USA']","{'Uruguay': '2 January 1987', 'Sweden': '3 Oct...","['Cameron, James (I)']","['Carroll, Gordon', 'Giler, David', 'Hill, Wal...","['Horner, James']","{'Armstrong, William (I)': 'Lydecker', 'Biehn,..."
74468,30189,Aliens of the Deep,2005.0,movie,['Color'],['Documentary'],['English'],['USA'],"{'Hungary': '20 September 2005', 'Argentina': ...","['Cameron, James (I)', 'Quale, Steven']","['Debiec, Christopher A.', 'Marsh, Ed W.', 'Ca...","['Hwang, Jeehun']","{'Atkins, Dr. Michael': 'Himself, Science obse..."
79965,236635,Hercules: The Legendary Journeys,1995.0,TelevisionSeries,['Color'],"['Drama', 'Adventure', 'Action', 'Fantasy']",['English'],"['New Zealand', 'USA']","{'Portugal': '8 February 1997', 'Netherlands':...","['Beesley, Mark (I)', 'Bond, Timothy', 'Braver...","['Joyce, Bernadette', 'Bielak, Robert', 'Gruen...","['LoDuca, Joseph']","{'Ackerley, Rachel': 'Evander', 'Acorn, Jonath..."


In [202]:
answer.count()

docid           14
title           14
year            14
type            14
colors          14
genres          14
languages       14
countries       14
releasedates    14
directors       14
producers       14
composers       14
cast            14
dtype: int64

In [208]:
sol=jaccard_index(result,answer)
Jaccardy_indexes_for_all_queries[query]=sol

In [209]:
acc=accuracy(Jaccardy_indexes_for_all_queries)
Jaccardy_indexes_for_all_queries

{"Give me all 'Batman' movies in French": 1.0,
 'Give any Woody Allen movies': 0.08433734939759036,
 'Which are the thriller movies directed by Nolan Christopher': 1.0,
 'Give me a drama movie in Hindi produced by Sanjay Leela Bhansali shot in India': 1.0,
 'Which are the drama and romance movies starring Morgan Freeman? ': 1.0,
 'give any Morgan Freeman composed by Nelson Riddle ': 1.0,
 'Give any movie having director as actor': 0.030504117924775743,
 "Give me a movie shot in Canada in the year 2006 having 'Love' in the title": 1.0,
 'Which are the movies directed by Sanjay Leela Bhansali and composed by Monty Sharma': 1.0,
 "Give me all 'Batman' movies": 1.0,
 'Give me a movie in English directed by James Cameron': 1.0}

In [215]:
#accuracy

acc

0.8286219515747605